### Vector Space Model

In [1]:
from nlp.data import load_20news

In [2]:
data = load_20news()

In [3]:
documents = data.data[:1_000]

## Tokenization

In [4]:
import spacy
nlp = spacy.load("en_core_web_lg")